In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score , mean_squared_error , mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression , Ridge , Lasso
from sklearn.model_selection import RandomizedSearchCV , train_test_split

In [5]:
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [6]:
df = pd.read_csv(".\data\StudentsPerformance.csv")
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [7]:
# separation of independent and dependent features 

X = df.drop(["math score"] , axis=1)
y = df["math score"]

In [10]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include = "object").columns

from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import ColumnTransformer

num_ss = StandardScaler()
cat_ohe = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder" ,cat_ohe , cat_features ),
        ("StandardScaler" , num_ss , num_features),
    ]
)

In [13]:
X = preprocessor.fit_transform(X)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [14]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [15]:
X.shape

(1000, 19)

In [16]:
X.columns

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [17]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size= 0.25 , random_state=42)

In [18]:
X_train.shape , X_test.shape

((750, 19), (250, 19))

In [19]:
def evaluate_model(true , predicted):
    mae = mean_absolute_error(true , predicted)
    mse = mean_squared_error(true , predicted)
    rmse = np.sqrt(mse)
    score  =r2_score(true , predicted)
    return mse , mae , rmse , score

In [20]:
models = {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    'Ridge' : Ridge(),
    "K Neighbours Regressor" : KNeighborsRegressor(),
    "Decision Tree" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGB Regressor" : XGBRegressor(),
    "Cat Boosting Regressor" : CatBoostRegressor(verbose=False),
    "Ada Boost regressor" : AdaBoostRegressor()
}
model_list =[]
r2_score_list = []
